#HW2 Problem:  LDA and QDA

Problem:  given a tabular dataset of feature values $X$ and classes $\boldsymbol{y}$, derive both LDA and QDA models, and determine how accurate they are.

More specifically, develop a program that reads in a numeric table with X and y from stdin, determines the normal distribution parameters for X, and derive the LDA and QDA models described in Chapter 2 of Murphy's book.

The columns of $X$ should all be numeric.  The values in the last column, $\boldsymbol{y}$,
can be either symbolic or numeric.
For example, with the <tt>iris</tt> dataset, the last column is symbolic.

Your program should print the "confusion matrix" for LDA, and for QDA.

If there are $k$ different classes, the confusion matrix is a $k \times k$ table
whose $i,j$-th entry is the number of times that an input row $\boldsymbol{x}$
was classified (by LDA or QDA) as being in class $i$, when in fact its $y$ value is $j$.

Your program should print the confusion matrices both for LDA and for QDA.
For example, with the iris dataset (as described below) the confusion matrices are 3x3, and your output should look like this:

<pre>
50 0 0
0 48 1
0 2 49
50 0 0
0 48 1
0 2 49
</pre>

There are implementations of LDA and QDA in the MASS package, and you can use them.




In [61]:
### %load_ext rmagic
### import rpy2 as Rpy

%load_ext rpy2.ipython

# new feature -- ipython is now part of rpy2.

##  Centroids of the 3 classes in the iris dataset

The $X$ matrix is of size $150 \times 4$, with 4 features
(Sepal Length, Sepal Width, Petal Length, Petal Width). The $150 \times 1$ vector $\boldsymbol{y}$ gives classes for these irises.
Each of the 3 classes has 50 observations.

We can compute the centroids of each class (means of each gaussian) using R.


In [62]:
%%R

data(iris)

X = data.matrix(iris[,1:4]) #  equivalently:  X = data.matrix(iris[, -5 ])
y = unclass(iris[,5])       #  equivalently:  y = unclass(iris$Species)

# print summary statistics for each kind of iris

print( by( X, y, summary ) )  # summarizes the subsets of X "grouped by" y

# the Mean values in these summaries give the centroids of each cluster/MVN


INDICES: 1
  Sepal.Length    Sepal.Width     Petal.Length    Petal.Width   
 Min.   :4.300   Min.   :2.300   Min.   :1.000   Min.   :0.100  
 1st Qu.:4.800   1st Qu.:3.200   1st Qu.:1.400   1st Qu.:0.200  
 Median :5.000   Median :3.400   Median :1.500   Median :0.200  
 Mean   :5.006   Mean   :3.428   Mean   :1.462   Mean   :0.246  
 3rd Qu.:5.200   3rd Qu.:3.675   3rd Qu.:1.575   3rd Qu.:0.300  
 Max.   :5.800   Max.   :4.400   Max.   :1.900   Max.   :0.600  
------------------------------------------------------------ 
INDICES: 2
  Sepal.Length    Sepal.Width     Petal.Length   Petal.Width   
 Min.   :4.900   Min.   :2.000   Min.   :3.00   Min.   :1.000  
 1st Qu.:5.600   1st Qu.:2.525   1st Qu.:4.00   1st Qu.:1.200  
 Median :5.900   Median :2.800   Median :4.35   Median :1.300  
 Mean   :5.936   Mean   :2.770   Mean   :4.26   Mean   :1.326  
 3rd Qu.:6.300   3rd Qu.:3.000   3rd Qu.:4.60   3rd Qu.:1.500  
 Max.   :7.000   Max.   :3.400   Max.   :5.10   Max.   :1.800  
-------------

In [63]:
%%R

# printing only the mean value/centroid for each class

classes = as.character( unique(iris$Species) )
k = length(classes)

for (j in 1:k) {
    cat( classes[j], "\n" )
    Xc = subset( X, (y == j) )
    print( apply(Xc, 2, mean) ) # vector of means for each column
}

setosa 
Sepal.Length  Sepal.Width Petal.Length  Petal.Width 
       5.006        3.428        1.462        0.246 
versicolor 
Sepal.Length  Sepal.Width Petal.Length  Petal.Width 
       5.936        2.770        4.260        1.326 
virginica 
Sepal.Length  Sepal.Width Petal.Length  Petal.Width 
       6.588        2.974        5.552        2.026 


##  Sample use of LDA and QDA -- with the iris dataset

The $X$ matrix is of size $150 \times 4$, with 4 features
(Sepal Length, Sepal Width, Petal Length, Petal Width).
Thus each column in this dataset is a random sample from a different distribution,
but they are nontrivially correlated.

The $150 \times 1$ vector $\boldsymbol{y}$ gives classes for these irises.
Each of the 3 classes has 50 observations.


In [64]:
%%R

# load the MASS package, which includes simple implementations of LDA and QDA

not.installed <- function(pkg) !is.element(pkg, installed.packages()[,1])

if (not.installed("MASS"))  install.packages("MASS")  # we need the MASS package

library(MASS)  #  load the MASS package

#  ?lda      #  help for the LDA classifier
#  ?qda      #  help for the QDA classifier


##run LDA on the iris data

In [65]:
%%R

X = data.matrix(iris[,1:4]) #  equivalently:  X = data.matrix(iris[, -5 ])
y = unclass(iris[,5])       #  equivalently:  y = unclass(iris$Species)

LDA.model <- lda(y ~ X)

## equivalently:
#  LDA.model <- qda(Species ~ ., iris)

cat("\n\nCentroids of the model:\n\n")
print( LDA.model$means )

cat("\n\nScaling of the model:\n\n")
print( LDA.model$scaling )

cat("\n\nWhat the model data structure looks like:\n\n")
str(LDA.model)   #  str() lets us inspect all the information in the model data structure




Centroids of the model:

  XSepal.Length XSepal.Width XPetal.Length XPetal.Width
1         5.006        3.428         1.462        0.246
2         5.936        2.770         4.260        1.326
3         6.588        2.974         5.552        2.026


Scaling of the model:

                     LD1         LD2
XSepal.Length  0.8293776  0.02410215
XSepal.Width   1.5344731  2.16452123
XPetal.Length -2.2012117 -0.93192121
XPetal.Width  -2.8104603  2.83918785


What the model data structure looks like:

List of 10
 $ prior  : Named num [1:3] 0.333 0.333 0.333
  ..- attr(*, "names")= chr [1:3] "1" "2" "3"
 $ counts : Named int [1:3] 50 50 50
  ..- attr(*, "names")= chr [1:3] "1" "2" "3"
 $ means  : num [1:3, 1:4] 5.01 5.94 6.59 3.43 2.77 ...
  ..- attr(*, "dimnames")=List of 2
  .. ..$ : chr [1:3] "1" "2" "3"
  .. ..$ : chr [1:4] "XSepal.Length" "XSepal.Width" "XPetal.Length" "XPetal.Width"
 $ scaling: num [1:4, 1:2] 0.8294 1.5345 -2.2012 -2.8105 0.0241 ...
  ..- attr(*, "dimnames")=List o

## predict() is used to turn a model into a function

If $M$ is a model, and $X$ is a (possibly new) set of $X$ values, then  <b>predict(M, X)</b> yields the vector of $\boldsymbol{y}$ values predicted by the model for the input matrix $X$.

If $M$ is a classification model, then <b>predict(M, X)</b> yields the classifications for feature vectors in the rows of $X$.

In [66]:
%%R

LDAclassifier = function(Model,X)  {
   predict(Model, as.data.frame(X))$class
}

# Compute the LDA predictions ("classifications") for each input row.

LDA.classifications = LDAclassification(LDA.model, X)

# Find all points whose classifications didn't agree with LDA.

LDA.disagreements = (1:nrow(X))[ LDA.classifications != y ]
print(LDA.disagreements)  # print row numbers where LDA differed from y


[1]  71  84 134


In [67]:
%%R

# Tabulate the number of LDA classification values vs. y values

LDA.confusion.matrix = table( LDA.classifications, y )

rownames( LDA.confusion.matrix ) = classes
colnames( LDA.confusion.matrix ) = classes
print( LDA.confusion.matrix )


                   y
LDA.classifications setosa versicolor virginica
         setosa         50          0         0
         versicolor      0         48         1
         virginica       0          2        49


##Print the confusion matrix in the format required by Mooshak

In [68]:
%%R

print_matrix_for_Mooshak = function(Matrix) {
    for (i in 1:nrow(Matrix)) {
       cat( Matrix[i,], "\n" )  # print each row as a sequence
    }
}


LDA.confusion.matrix = table( LDA.classifications, y )

print_matrix_for_Mooshak( LDA.confusion.matrix )


50 0 0 
0 48 1 
0 2 49 


##run QDA on the iris data

In [69]:
%%R

QDA.model <- qda(y ~ X)

## equivalently:
#  QDA.model <- qda(Species ~ ., iris)

cat("\n\nCentroids of the model:\n\n")
print( QDA.model$means )

cat("\n\nScaling of the model:\n\n")
print( QDA.model$scaling )

cat("\n\nWhat the model object looks like:\n\n")
str(QDA.model)   #  str() lets us inspect all the information in the model data structure

QDAclassification = function(Model,X)  {
   predict(Model,as.data.frame(X))$class
}

# find all points whose classifications didn't agree with QDA

cat("\n\nNumbers of misclassified instances with the Quadratic Discriminant:\n\n")

QDA.disagreements = (1:nrow(X))[ QDAclassification(QDA.model, X) != y ]
cat(QDA.disagreements, "\n\n")

cat("\n\nData for the instances misclassified with the Quadratic Discriminant:\n\n")

print(iris[QDA.disagreements,])




Centroids of the model:

  XSepal.Length XSepal.Width XPetal.Length XPetal.Width
1         5.006        3.428         1.462        0.246
2         5.936        2.770         4.260        1.326
3         6.588        2.974         5.552        2.026


Scaling of the model:

, , 1

                      1         2          3          4
XSepal.Length -2.836962 -3.145110 -0.8878372 -0.4637981
XSepal.Width   0.000000  3.938634  0.1263223 -0.2043238
XPetal.Length  0.000000  0.000000  5.9785398 -1.7416275
XPetal.Width   0.000000  0.000000  0.0000000 10.2978593

, , 2

                      1         2         3          4
XSepal.Length -1.937342  1.197909  1.958819 -0.6910239
XSepal.Width   0.000000 -3.746750  1.150301  2.0855780
XPetal.Length  0.000000  0.000000 -3.389213  2.8839194
XPetal.Width   0.000000  0.000000  0.000000 -9.3404922

, , 3

                     1          2          3          4
XSepal.Length 1.572625 -0.8085097  2.6909994  0.4068814
XSepal.Width  0.000000  3.4866013 

##That's it!  Now just write an R script that does this for any dataset it reads in.

To get you started, your R script can be an extension of this outline:

<pre>
#  Read in a table (in csv format) from standard input:
Table = read.csv( file("stdin"), header=TRUE )

X = data.matrix( Table[, 1:(ncol(Table)-1) ]
classifications = Table[, ncol(Table) ]

y = unclass(classifications)  # convert the class values into numeric indices

n = nrow(X)
p = ncol(X)

# ... construct an LDA representation of X
# ... determine for which rows in X the LDA classification differs from y
# ... print the confusion matrix for LDA

# ... construct a QDA representation of X
# ... determine for which rows in X the QDA classification differs from y
# ... print the confusion matrix for QDA
</pre>



#What your program's output should look like

For example, with the iris dataset, the confusion matrices are $3 \times 3$, and it turns out the LDA and QDA matrices are identical.
Thus the output should look like this:

<pre>
50 0 0
0 48 1
0 2 49
50 0 0
0 48 1
0 2 49
</pre>